In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [44]:
class MNISTClassifier(nn.Module):
  def __init__(self):
    super(MNISTClassifier, self).__init__()
    self.layer1 = nn.Linear(28*28, 128)
    self.layer2 = nn.Linear(128, 64)
    self.layer3 = nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = F.relu(self.layer1(x))
    x = F.relu(self.layer2(x))
    x = F.softmax(self.layer3(x), dim=1)
    return x

model = MNISTClassifier()

In [45]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [47]:
model = MNISTClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/200:.4f}')
            running_loss = 0.0

Epoch [1/5], Step [200/938], Loss: 1.7973
Epoch [1/5], Step [400/938], Loss: 1.6452
Epoch [1/5], Step [600/938], Loss: 1.5846
Epoch [1/5], Step [800/938], Loss: 1.5756
Epoch [2/5], Step [200/938], Loss: 1.5543
Epoch [2/5], Step [400/938], Loss: 1.5516
Epoch [2/5], Step [600/938], Loss: 1.5519
Epoch [2/5], Step [800/938], Loss: 1.5405
Epoch [3/5], Step [200/938], Loss: 1.5357
Epoch [3/5], Step [400/938], Loss: 1.5311
Epoch [3/5], Step [600/938], Loss: 1.5271
Epoch [3/5], Step [800/938], Loss: 1.5282
Epoch [4/5], Step [200/938], Loss: 1.5212
Epoch [4/5], Step [400/938], Loss: 1.5188
Epoch [4/5], Step [600/938], Loss: 1.5180
Epoch [4/5], Step [800/938], Loss: 1.5210
Epoch [5/5], Step [200/938], Loss: 1.5116
Epoch [5/5], Step [400/938], Loss: 1.5167
Epoch [5/5], Step [600/938], Loss: 1.5133
Epoch [5/5], Step [800/938], Loss: 1.5153


In [48]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [49]:
accuracy = 100 * correct / total
print(f'\nTest Accuracy: {accuracy:.2f}%')


Test Accuracy: 95.02%
